# Lesson 1: Tensorflow Basics

In [2]:
# @title imports!
# !pip install 

import torch
import numpy as np

In [10]:
# basic tensors
t1 = torch.tensor(4.) # float32
t2 = torch.tensor([1, 2, 3, 4]) # vector
t3 = torch.tensor([[5., 6.], [7., 8.], [9., 10.]]) # matrix

# tensor operations
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
y = w * x + b
# print(y)

# compute derivatives
y.backward()

# display gradients
print("dy/dx:", x.grad)
print("dy/dw:", w.grad) # has the same value as x, ie. 3 
print("dy/db:", b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


In [ ]:
# inter-op with numpy

a1 = np.array([[1, 2], [3, 4]])
ten1 = torch.from_numpy(x)

# Lesson 2 - Linear Regression

In [12]:
import numpy as np
import torch

In [13]:
# input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
# print(inputs)
# print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [19]:
# linear regression model from scratch

# weights and biases
w = torch.randn(2, 3, requires_grad=True) # 2 row, 3 column matrix of random values
b = torch.randn(2, requires_grad=True) # 2 row, 1 column matrix of random values

**note:** random values are picked from a normal distribution with mean 0 and standard deviation 1.

In [20]:
# linear regression model, simply a Y = mX + C thing
def model(x):
  return x @ w.t() + b # (@ represents matrix multiplication)

# generate predictions
preds = model(inputs)
print(preds)

tensor([[ 60.3191, -27.1181],
        [ 78.6981, -51.4230],
        [ 74.2860, -69.9580],
        [ 76.5674,  20.0796],
        [ 66.8297, -84.0901]], grad_fn=<AddBackward0>)


In [16]:
print(targets) # compare our prediction with the target values

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [23]:
# loss function (mean squared error - mse)
def mse(t1, t2):
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()

loss = mse(preds, targets)
print("loss:", loss)

# compute gradients of the loss wrt. weights and biases
loss.backward()
print(w)
print(w.grad) # gradients for weights

loss: tensor(12169.7676, grad_fn=<DivBackward0>)
tensor([[ 0.6309,  0.0023,  0.3322],
        [ 0.9101, -0.5100, -1.4275]], requires_grad=True)
tensor([[  -142.8349,  -1406.1621,   -613.5878],
        [-10871.3105, -13468.1465,  -8109.0142]])


In [26]:
# modify weights and biases
with torch.no_grad():
  w -= w.grad * 1e-5 # 1e-5 to make sure the weights aren't updated too much at a time
  b -= b.grad * 1e-5

loss = mse(preds, targets)
print("new loss:", loss) # let's check that the loss is lower!

# reset gradients to 0 before proceeding
w.grad.zero_()
b.grad.zero_()
print(w.grad, b.grad) # the gradients are reset!

new loss: tensor(12169.7676, grad_fn=<DivBackward0>)
tensor([[0., 0., 0.],
        [0., 0., 0.]]) tensor([0., 0.])


In [27]:
# @title train the model using gradient descent!

preds = model(inputs)
print(preds)

# calc loss
loss = mse(preds, targets)
print(loss) # @markdown the loss should be better than last time!

tensor([[64.2500, 34.2258],
        [83.9785, 29.3849],
        [81.3794, 26.6718],
        [79.4996, 79.7248],
        [72.4638, -5.7652]], grad_fn=<AddBackward0>)
tensor(4083.6875, grad_fn=<DivBackward0>)


In [32]:
# @title training for multiple epochs
for i in range(10000):
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

    w.grad.zero_()
    b.grad.zero_()
  
# and then calculate the loss
preds = model(inputs)
loss = mse(preds, targets)
print("final preds:\n", preds)
print("final loss:\n", loss)

final preds:
 tensor([[ 57.1353,  70.6433],
        [ 82.2256, 100.4605],
        [118.6980, 132.9042],
        [ 21.0852,  36.9732],
        [101.9156, 119.1914]], grad_fn=<AddBackward0>)
final loss:
 tensor(0.5646, grad_fn=<DivBackward0>)
